In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd

### Uploading My Data

The following is where I load the healthy corner stores and farmers markets datasets from my Initial_Exploration_and_Cleaning folder and merge them into one combined dataset called all_healthy_counts_df as I did in my All Healthy Food Locations (Interactive) notebook. 


In [2]:
healthy_corner_stores_geo_df=gpd.read_file('../../data/Raw_Data/PhillyHealth_Healthy_corner_stores.geojson')
cleaned_healthycornerstores_df=healthy_corner_stores_geo_df[['STORE_ADDR', 'OFFICIAL_S', 'ZIP', 'geometry']]
cleaned_healthycornerstores_df.columns = ['Address', 'Name', 'Zipcode', 'geometry']
cleaned_healthycornerstores_df['longitude']=cleaned_healthycornerstores_df['geometry'].x
cleaned_healthycornerstores_df['latitude']=cleaned_healthycornerstores_df['geometry'].y

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
farmers_markets_geo_df=gpd.read_file('../../data/Raw_Data/Farmers_Markets.geojson')
cleaned_farmers_markets_geo_df=farmers_markets_geo_df[['ADDRESS', 'NAME', 'ZIP', 'geometry']]
cleaned_farmers_markets_geo_df.columns = ['Address', 'Name', 'Zipcode', 'geometry']
cleaned_farmers_markets_geo_df['longitude']=cleaned_farmers_markets_geo_df['geometry'].x
cleaned_farmers_markets_geo_df['latitude']=cleaned_farmers_markets_geo_df['geometry'].y

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [4]:
all_healthy_counts_df=pd.concat([cleaned_farmers_markets_geo_df, cleaned_healthycornerstores_df], axis=0)
all_healthy_counts_df.head()

,Address,Name,Zipcode,geometry,longitude,latitude
0,2501 E. Cumberland Ave,Greensgrow Farm Stand,19125,POINT (-75.1215584811801 39.97843093078643),-75.121558,39.978431
1,Diamond St and N. Howard St,Norris Square Neighborhood Project,19122,POINT (-75.13392516927033 39.98177864369919),-75.133925,39.981779
2,18th and Walnut St,Rittenhouse,19103,POINT (-75.17067028026975 39.95014093264241),-75.170670,39.950141
3,22nd St. and Fairmount Ave.,Fairmount,19130,POINT (-75.17398597335172 39.9673232257184),-75.173986,39.967323
4,1628-32 Ridge Ave,Francisville Artisan and Farmers Market,19130,POINT (-75.1640490804737 39.96981974381605),-75.164049,39.969820


### Finding the Number of Healthy Store Locations in Each Zipcode

I would like to calculate the number of stores in each zipcode so that I can eventually create a new dataset which includes the Diabetes Counts, Median Income, Healthy Store Count, Unhealthy Store Count, and Ratio of Healthy to Unhealthy store counts for each Zipcode. 

In [5]:
all_healthy_counts_df['Zipcode'].nunique()

63

There are clearly Zipcodes in the wrong format since Philly only has 48 zipcodes. 

In [6]:
all_healthy_counts_df['Zipcode'].unique()

array([19125, 19122, 19103, 19130, 19107, 19147, 19106, 19144, 19129,
       19118, 19123, 19140, 19141, 19133, 19121, 19124, 19149, 19116,
       19128, 19119, 19138, 19127, 19148, 19145, 19143, 19139, 19104,
       19151, 19131, '19146', '19121', '19132', '19140', '19120', '19145',
       '19142', '19134', '19133', '19130', '19139', '19124', '19143',
       '19131', '19137', '19135', '19125', '19147', '19144', '19122',
       '19148', '19111', '19141', '19151', '19104', '19138', '19123',
       '19149', '19152', '19136', '19128', '19115', '19126', '19129'],
      dtype=object)

They are not all in string form so I need to convert them first. This may explain why there are 63 zipcodes accounted for rather than up to 48. 

In [7]:
all_healthy_counts_df['Zipcode']=all_healthy_counts_df['Zipcode'].apply(str)

In [8]:
all_healthy_counts_df['Zipcode'].unique()

array(['19125', '19122', '19103', '19130', '19107', '19147', '19106',
       '19144', '19129', '19118', '19123', '19140', '19141', '19133',
       '19121', '19124', '19149', '19116', '19128', '19119', '19138',
       '19127', '19148', '19145', '19143', '19139', '19104', '19151',
       '19131', '19146', '19132', '19120', '19142', '19134', '19137',
       '19135', '19111', '19152', '19136', '19115', '19126'], dtype=object)

In [9]:
all_healthy_counts_df['Zipcode'].nunique()

41

Now that I have converted all my Zipcodes to the correct string format, I can see that there are now actually 41 unique zipcodes. 

In [10]:
allzip=all_healthy_counts_df['Zipcode'].value_counts()

In [11]:
allzip

19134    68
19132    47
19140    47
19143    46
19120    43
19148    41
19133    39
19121    35
19139    30
19146    27
19124    27
19142    26
19145    24
19144    22
19131    21
19122    19
19147    18
19138    15
19151    14
19123    14
19130    13
19104     9
19141     9
19125     9
19111     7
19128     7
19135     7
19136     7
19126     6
19129     4
19149     4
19137     2
19103     2
19152     2
19106     2
19127     1
19115     1
19119     1
19116     1
19118     1
19107     1
Name: Zipcode, dtype: int64

Just to make sure that these 41 unique zipcodes account for all of the data in this all_healthy_counts_df, I will add all of the value counts to ensure that all 719 store counts are accounted for in these 41 zipcodes. 

In [12]:
sum(allzip)

719

Now I need to write each of these value counts as a list so I can add it as a column to my eventual dataset which combines diabetes, income, healthy store counts, unhealthy store counts, and ratio for each zipcode. To make sure I write the value counts in the correct order in my list, I will sort the zipcodes by low to higher since that is how the zipcodes are ordered in my diabetes and income datasets. 

In [13]:
allzip.sort_index()

19103     2
19104     9
19106     2
19107     1
19111     7
19115     1
19116     1
19118     1
19119     1
19120    43
19121    35
19122    19
19123    14
19124    27
19125     9
19126     6
19127     1
19128     7
19129     4
19130    13
19131    21
19132    47
19133    39
19134    68
19135     7
19136     7
19137     2
19138    15
19139    30
19140    47
19141     9
19142    26
19143    46
19144    22
19145    24
19146    27
19147    18
19148    41
19149     4
19151    14
19152     2
Name: Zipcode, dtype: int64

In [14]:
Healthy_Store_Counts = [0, 2, 9, 2, 1, 0, 7, 0, 0, 1, 1, 1, 1, 43, 35, 19, 14, 27, 9, 6, 1, 7, 4, 13, 21, 47, 39, 68, 7, 7, 2, 15, 30, 47, 9, 26, 46, 22, 24, 27, 18, 41, 4, 0, 14, 2, 0, 0]

### About This Data

* This list titled Healthy_Store_Counts includes all of the values for store counts for each zipcode from 19102 to 19153. I've included 0s in the zipcodes that don't have any locations according to my all_healthy_counts_df so that when I merge it to my final dataset, it will align with the zipcodes in the diabetes and income dataset. 